#### Job Shop Scheduling ####

**Short summary**

1. The problem consists of N *jobs* 1..N  each job has some number of *tasks*
1. For each job, the tasks must be done in sequence
1. Each task requires a *resource*,  and two tasks that use the same resource cannot be scheduled at the same time
1. All tasks take three time units to complete
1. All tasks are ready for execution at time 0, and must complete at or before time 15
  1. Which means that if any job has more five tasks, it cannot be scheduled, no matter what

There are some hints at the bottom of the file.

Input to the problem comes in a variable like this:


In [172]:
# Job 1
#   (o11, r1)   (o12, r2) (o13, r3)
# Job 2
#   (o21, r1)  (o22, r2)
# Job 3
#   (o31, r3) (o32, r1) (o33, r2)
# Job 4
#   (o41, r4) (o42, r2)
#
# The ri are the resources

example = [[("o11", "r1"), ("o12", "r2"), ("o13", "r3")],
           [("o21", "r1"), ("o22", "r2")],
           [("o31", "r3"), ("o32", "r1"), ("o33", "r2")],
           [("o41", "r4"), ("o42", "r2")]]


----------------------------------------------

In [173]:
from constraint import *

In [174]:
###  Your implementation of solveCSP here:  takes a problem description as input, returns a 
###  list of solutions as output.  The method problem.getSolutions() returns a list of solutions.
###  Each solution is a dictionary -- key is a task, value is the value it is assigned by the CSP
def solveCSP(problemList):
    problem = Problem()
    #print(timeList)
    tasks = []
    for jobno in range(len(problemList)): 
        task = subtasks(problemList, jobno)
        for subtask in task:
            problem.addVariable(subtask, range(5))
    orderCSP(problem, problemList)
    mutexCSP(problem, problemList)
    return problem.getSolutions()    

In [175]:
def orderCSP(problem, problemList):
    for jobno in range(len(problemList)):
        subtasks = problemList[jobno]
        for i in range(len(subtasks)-1):
            task1 = subtasks[i]
            task2 = subtasks[i+1]
            addConstraintByOrder(problem, (task1[0],task2[0]))               

In [176]:
def addConstraintByOrder(problem, pair):
    problem.addConstraint(lambda x, y: x < y, pair)

In [177]:
def mutexCSP(problem, problemList):
    subtasksList = []
    for jobno in range(len(problemList)):
        for subtask in getSubTask(problemList, jobno):
            subtasksList.append(subtask)
    
    for i in range(len(subtasksList)):
        for j in range(i+1, len(subtasksList)):
            if subtasksList[i][1] == subtasksList[j][1]:
                addConstraintByMutex(problem, (subtasksList[i][0],subtasksList[j][0]))                

In [178]:
def getSubTask(problemList, jobno):
    return problemList[jobno]

In [179]:
def addConstraintByMutex(problem, pair):
    problem.addConstraint(lambda x, y: x != y, pair)

In [180]:
def printSolution(problemList):
    solutions = solveCSP(problemList)
    if len(solutions) == 0:
        print("No solutions for this problem")
    else:
        print("There are " + str(len(solutions)) + " solutions.  Here is one:\n")
        solution = solutions[0]
        for jobno in range(0, len(problemList)):
            print("Schedule for Job " + str(jobno+1))
            for subtask in sorted(subtasks(problemList, jobno)):
                print("   Subtask " + subtask + " starts at " + str(solution[subtask]*3) + \
                      " ends at " + str(2 + int(solution[subtask]*3)) + "\tuses " + resourceFor(subtask, problemList))

In [181]:
def subtasks(problemList, jobno):
    subtasksList = problemList[jobno]
    taskList = [subtask[0] for subtask in subtasksList]
    return taskList

In [182]:
def resourceFor(subtask, problemList):
    resource = ""
    for jobno in range(len(problemList)):
        subtasks = problemList[jobno]
        for task in subtasks:
            if task[0] == subtask:
                resource = task[1]
    return resource

In [183]:
printSolution(example)

There are 325 solutions.  Here is one:

Schedule for Job 1
   Subtask o11 starts at 6 ends at 8	uses r1
   Subtask o12 starts at 9 ends at 11	uses r2
   Subtask o13 starts at 12 ends at 14	uses r3
Schedule for Job 2
   Subtask o21 starts at 9 ends at 11	uses r1
   Subtask o22 starts at 12 ends at 14	uses r2
Schedule for Job 3
   Subtask o31 starts at 0 ends at 2	uses r3
   Subtask o32 starts at 3 ends at 5	uses r1
   Subtask o33 starts at 6 ends at 8	uses r2
Schedule for Job 4
   Subtask o41 starts at 0 ends at 2	uses r4
   Subtask o42 starts at 3 ends at 5	uses r2


-----------------------------------

**Hints**
1.  Since all tasks take three time units, it's easiest to partition time into "buckets" each of which is three time units long.  For example, bucket 1 is the interval between t=0 and t=2 inclusive, bucket 1 is t=3 to t=5 inclusive.  You will notice from the **printSolution** code that my solver does that bucketing, then when it prints the solution it converts the time bucket number to a real point in time
2.  There are only two kinds of constraints:  order constraints (one task must come after another) and mutex constraints (two tasks must not be scheduled at the same time).   If you write helper functions for those two constraints, it will simplify your code a lot.